In [1]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
from fuzzywuzzy import process

# Function to fix spelling mistakes using fuzzy matching
def fix_spelling(word, choices):
    best_match, score = process.extractOne(word, choices)
    if score >= 90:  # Adjust threshold as needed
        return best_match
    else:
        return word

# Function to clean and fix the data
def clean_data(df):
    # Remove commas from agency names
    df['Department/Ind.Agency'] = df['Department/Ind.Agency'].apply(lambda x: re.sub(r'(\b[A-Z\s&.]+),([A-Z\s&.]+\b)', r'\1\2', x))
    df['Awardee'] = df['Awardee'].astype(str)
    df['Awardee'] = df['Awardee'].apply(lambda x: re.sub(r'(\b[A-Z\s&.]+),([A-Z\s&.]+\b)', r'\1\2', x))
    
    # Fix spelling mistakes and incomplete words
    df['Department/Ind.Agency'] = df['Department/Ind.Agency'].apply(lambda x: re.sub(r'\bDEPT OF\b', 'DEPARTMENT OF', x))
    df['Department/Ind.Agency'] = df['Department/Ind.Agency'].apply(lambda x: re.sub(r'\bAGRICULTUR[A-Za-z]+\b', 'AGRICULTURE', x))
    df['Department/Ind.Agency'] = df['Department/Ind.Agency'].apply(lambda x: re.sub(r'\bVETERANS AFFAIRS\b', 'VETERANS AFFAIRS', x))
    df['Department/Ind.Agency'] = df['Department/Ind.Agency'].apply(lambda x: re.sub(r'\bDEPARTMENT[A-Za-z]*\b', 'Department/Ind.Agency', x))
    df['Department/Ind.Agency'] = df['Department/Ind.Agency'].apply(lambda x: re.sub(r'\bHOMELAND SECURITY\b', 'HOMELAND SECURITY', x))
    df['Department/Ind.Agency'] = df['Department/Ind.Agency'].apply(lambda x: re.sub(r'\bINTERIOR\b', 'INTERIOR', x))
    
    # Fix spelling mistakes using fuzzy matching
    choices = ['DEFENSE', 'AGRICULTURE', 'VETERANS', 'AFFAIRS', 'HOMELAND', 'SECURITY', 'INTERIOR']
    df['Department/Ind.Agency'] = df['Department/Ind.Agency'].apply(lambda x: fix_spelling(x, choices))
    
    # Split the data into department and awardee parts
    df[['Awardee_Name', 'Awardee_Location']] = df['Awardee'].str.extract(r'(.*),\s*(.*)')

    # Normalize department names
    df['Department/Ind.Agency'] = df['Department/Ind.Agency'].str.strip().str.replace('DEPARTMENT OF', '').str.strip()
    
    return df


# Convert the sample data to a DataFrame
df = pd.read_csv(r"C:\Users\dgarg\Downloads\Book2.csv")

# Clean the DataFrame
cleaned_df = clean_data(df)

# Print the cleaned DataFrame
print(cleaned_df)


C:\Users\dgarg\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


                                 Department/Ind.Agency  \
0                                             VETERANS   
1              TRANSPORTATION Department/Ind.Agency OF   
2                                              DEFENSE   
3                                              DEFENSE   
4                                          AGRICULTURE   
..                                                 ...   
144  HEALTH AND HUMAN SERVICES Department/Ind.Agenc...   
145                                            DEFENSE   
146                                            DEFENSE   
147                                            DEFENSE   
148                                        AGRICULTURE   

                                               Awardee Awardee_Name  \
0                                                null           NaN   
1                                                  nan          NaN   
2    AMERICAN BACKFLOW PREVENTION ASSOCIATION West ...          NaN   
3                  